In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm   #A package that provides a progress bar for tracking the progress of tasks.

In [2]:
import warnings
import torch.nn.functional as F  #imports the torch.nn.functional module from PyTorch. This module provides a set of functions that are commonly used in neural networks for various operations, 
#such as activation functions, loss functions, and other functional operations.

# Disable warnings
warnings.filterwarnings("ignore")

In [3]:
# Define the generator model
class Generator(nn.Module):
    def __init__(self, input_channels, output_channels, num_filters=64):  #three parameters #input and output channels of the generator and 
#         the number of filters used in the convolutional layers.
        super(Generator, self).__init__()

        # Encoder#The num_filters parameter allows you to adjust the capacity 
        #and complexity of the generator model. Increasing the number of filters
#         can potentially enable the model to capture more intricate and detailed features but also increases the computational 
# requirements and memory usage.
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, num_filters, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True), #argument is set for the ReLU or Leaky ReLU activation functions in PyTorch, it means that 
#             the operation is performed in-place. This means that the original tensor is modified directly without creating a new tensor. 
            nn.Conv2d(num_filters, num_filters * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_filters * 2, num_filters * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 4),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_filters * 4, num_filters * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 8),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_filters * 8, num_filters * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 8),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_filters * 8, num_filters * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 8), #Batch normalization is a technique that helps improve the training and performance of neural networks.
            nn.ReLU(inplace=True), #except for the last layer which uses only the ReLU activation function.
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(num_filters * 8, num_filters * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(num_filters * 8, num_filters * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(num_filters * 8, num_filters * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(num_filters * 4, num_filters * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(num_filters * 2, num_filters, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(num_filters, output_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(), #hyperbolic tangent (Tanh) activation function to produce pixel values in the range [-1, 1] in last layer
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [4]:
# Define the discriminator model
class Discriminator(nn.Module): #defines the architecture of the discriminator
    def __init__(self, input_channels, num_filters=64): #input channels and filters
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(input_channels, num_filters, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True), # -ve slope of 0.2,
            nn.Conv2d(num_filters, num_filters * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 2), #2D input feature maps.
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(num_filters * 2, num_filters * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 4),  #It operates on the 4D input tensor of shape (batch_size, num_channels, height, width)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(num_filters * 4, num_filters * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_filters * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(num_filters * 8, 1, kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.model(x)
        return output
# Leaky ReLU activation function is -infinity to infinity for positive inputs 
# and -infinity to alpha * x for negative inputs, where alpha is the slope parameter.


# The Leaky ReLU (Rectified Linear Unit) is an activation function commonly used in deep neural networks. It is similar 
# to the regular ReLU function but includes a small slope for negative input values, allowing a small gradient to flow through 
# when the input is negative. This helps alleviate the "dying ReLU" problem, 
# where neurons with negative inputs would output zero and stop updating their weights during training.

# f(x) = { x,              if x > 0,
#        { alpha * x,      if x <= 0,
# where alpha is a small positive slope parameter (typically a small value like 0.01).

In [5]:
# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
import os

# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [7]:
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [8]:
# # Set device (CPU or GPU)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyperparameters
lr = 0.05 #the rate at which a model adjusts its parameters during the training process.
num_epochs = 200  #epoch refers to a single pass of the entire training dataset 
batch_size = 1
sample_interval = 100  # Interval for saving generated images

In [9]:

# Initialize the generators and discriminators
G_AB = Generator(input_channels=3, output_channels=3).to(device)
G_BA = Generator(input_channels=3, output_channels=3).to(device)
D_A = Discriminator(input_channels=3).to(device) #using the Discriminator class
D_B = Discriminator(input_channels=3).to(device)


# Define loss function and optimizers
criterion_GAN = nn.MSELoss()  #adversarial loss (GAN loss) the generator to produce more realistic samples and discriminate between real and generated image
criterion_cycle = nn.L1Loss()  #from one domain to another domain # cycle-consistency loss

#the parameters to optimize are obtained by concatenating the parameters of G_AB and G_BA using the list() function.
optimizer_G = optim.Adam(
    list(G_AB.parameters()) + list(G_BA.parameters()), lr=lr, betas=(0.5, 0.999) #The betas represent the coefficients used for computing running averages of the gradient and its square.
)
optimizer_D = optim.Adam(
    list(D_A.parameters()) + list(D_B.parameters()), lr=lr, betas=(0.5, 0.999)
)


In [10]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# Define the path to your dataset
dataset_pathtrain = "C:/Users/PG LAB/Desktop/cyclegan data/A1" #HE Train
dataset_pathtest = "C:/Users/PG LAB/Desktop/cyclegan data/B1" #IHC train

# Define the transformation to apply to the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the image to a fixed size
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the image
])

# Load the dataset
datasettrain = ImageFolder(dataset_pathtrain, transform=transform)
datasettest = ImageFolder(dataset_pathtest, transform=transform)



# Assuming you have separate dataloaders for source and target datasets
source_dataloader = DataLoader(datasettrain, batch_size=batch_size, shuffle=False)
target_dataloader = DataLoader(datasettest, batch_size=batch_size, shuffle=False)

In [11]:
# Training loop
for epoch in range(num_epochs):
    for i, (source_images, target_images) in tqdm(enumerate(zip(source_dataloader, target_dataloader))):
        # Transfer images to the GPU if available
        source_images = source_images[0].to(device)
        target_images = target_images[0].to(device)
#         source_images = [image.to(device) for image in source_images]
#         target_images = [image.to(device) for image in target_images]

        # Adversarial ground truthslabels (valid and fake)
        valid = torch.ones(source_images.size(0), 1).to(device) 
        fake = torch.zeros(source_images.size(0), 1).to(device)
#         valid = torch.ones(source_images[0].size(0), 1).to(device)
#         fake = torch.zeros(source_images[0].size(0), 1).to(device)
        
        ### Train the generators ###
        optimizer_G.zero_grad()

        # Identity loss (optional)
        loss_id = criterion_cycle(G_AB(source_images), source_images)  # Identity loss for generator G_AB
        loss_id += criterion_cycle(G_BA(target_images), target_images)  # Identity loss for generator G_BA
        loss_id *= 0.5  # Weighting factor for the identity loss

        # Forward cycle loss #econstructed images (reconstructed_A) and the source images (source_images) 
        fake_B = G_AB(source_images)
        reconstructed_A = G_BA(fake_B)
        loss_cycle_A = criterion_cycle(reconstructed_A, source_images)  # Forward cycle loss for generator G_AB

        # Backward cycle loss
        fake_A = G_BA(target_images)
        reconstructed_B = G_AB(fake_A)
        loss_cycle_B = criterion_cycle(reconstructed_B, target_images)  # Backward cycle loss for generator G_BA

        # Total generator loss
        loss_G = loss_id + loss_cycle_A + loss_cycle_B

        # Update generators
        loss_G.backward()
        optimizer_G.step()

        ### Train the discriminators ###
        optimizer_D.zero_grad()

        # Real loss
        loss_real_A = criterion_GAN(D_A(source_images), valid)
        loss_real_B = criterion_GAN(D_B(target_images), valid)

        # Fake loss
        fake_A = G_BA(target_images)
        fake_B = G_AB(source_images)
        loss_fake_A = criterion_GAN(D_A(fake_A.detach()), fake)
        loss_fake_B = criterion_GAN(D_B(fake_B.detach()), fake)

        # Total discriminator loss
        loss_D_A = (loss_real_A + loss_fake_A) * 0.5
        loss_D_B = (loss_real_B + loss_fake_B) * 0.5
        loss_D = loss_D_A + loss_D_B

        # Update discriminators
        loss_D_A.backward()
        optimizer_D.step()

        #The fake loss measures the difference between the predictions of the discriminators for the generated images 
# (fake_A and fake_B) and the expected labels (fake). It is also computed using the MSE loss.
#----------------------------------------------------------------

        # Print progress
        batches_done = epoch * len(source_dataloader) + i
        if batches_done % sample_interval == 0:
            print(
                f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(source_dataloader)}] [D loss: {loss_D.item()}] [G loss: {loss_G.item()}]"
            )

    # Save generated images after each epoch
    with torch.no_grad():
        fake_B = G_AB(source_images)
        fake_A = G_BA(target_images)
        #"C:/Users/sidra/Desktop/bci small/generated_images"
        save_image(fake_B, "C:/Users/PG LAB/Desktop/cyclegan data/generated_images/fake_B_{epoch}.png", normalize=True)
        save_image(fake_A, "C:/Users/PG LAB/Desktop/cyclegan data/generated_images/fake_A_{epoch}.png", normalize=True)


1it [00:05,  5.11s/it]

[Epoch 0/200] [Batch 0/413] [D loss: 0.4751289188861847] [G loss: 2.108567237854004]


101it [00:31,  3.70it/s]

[Epoch 0/200] [Batch 100/413] [D loss: 0.7846940159797668] [G loss: 1.279509425163269]


201it [00:57,  3.74it/s]

...

214it [00:55,  3.88it/s]

[Epoch 199/200] [Batch 213/413] [D loss: 0.7655213475227356] [G loss: 1.1982523202896118]


314it [01:20,  4.05it/s]

[Epoch 199/200] [Batch 313/413] [D loss: 0.768184244632721] [G loss: 0.7204058766365051]


413it [01:46,  3.90it/s]


In [12]:
# Testing loop
# Save test images and the translated images
G_AB.eval()
G_BA.eval()
with torch.no_grad():
    for i, (source_images, target_images) in tqdm(enumerate(zip(source_dataloader, target_dataloader))):
        # Transfer images to the GPU if available
        source_images = source_images[0].to(device)
        target_images = target_images[0].to(device)

        # Generate images
        fake_B = G_AB(source_images)
        fake_A = G_BA(target_images)

        # Save images
        save_image(source_images, f"C:/Users/PG LAB/Desktop/cyclegan data/test_images/source_{i}.png", normalize=True)
        save_image(target_images, f"C:/Users/PG LAB/Desktop/cyclegan data/test_images/target_{i}.png", normalize=True)
        save_image(fake_B, f"C:/Users/PG LAB/Desktop/cyclegan data/fake_B_{i}.png", normalize=True)
        save_image(fake_A, f"C:/Users/PG LAB/Desktop/cyclegan data/test_images/fake_A_{i}.png", normalize=True)


413it [00:51,  7.96it/s]


In [13]:
G_AB.eval()

Generator(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(512, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (12): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): Conv2d(512, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
 

In [14]:
G_BA.eval()

Generator(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(512, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (12): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): Conv2d(512, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
 

warnings.filterwarnings("default")